<a href="https://colab.research.google.com/github/Nosa1323/useful_notebooks/blob/main/how_to_make_reports_rus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Как делать отчеты из транзакционных данных

by Gleb Mikhaylov, мой канал в тг https://t.me/mikhaylovgleb

Я буду использовать https://www.kaggle.com/carrie1/ecommerce-data . Я залил их на свой гугл диск, и тут скачиваю по ссылке, так что все работает сразу, можно поиграть, покрутить

https://drive.google.com/file/d/1AclnBOt1YoCoJTxNtIlhVhrPRD0b8DTB/view?usp=sharing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
! gdown --id 1AclnBOt1YoCoJTxNtIlhVhrPRD0b8DTB

Downloading...
From: https://drive.google.com/uc?id=1AclnBOt1YoCoJTxNtIlhVhrPRD0b8DTB
To: /content/ecommerce-data.zip
7.55MB [00:00, 35.2MB/s]


In [ ]:
df = pd.read_csv('ecommerce-data.zip',encoding='ISO-8859-1')

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Вот, тут транзакции по пользователям. Они покупают какие-то товары. Посмотрим какой тут временной промежуток. И переведем дату из строки в формат даты

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],format='%m/%d/%Y %H:%M')

In [ ]:
df['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [ ]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

Примерно год. Теперь я хочу сделать отчет по продажам на каждый месяц.

# 1. Создай таблицу с уникальными юзерами

В реальной жизни такая таблица обычно имеется сразу и можно ей пользоваться. Здесь у нас такой нет -- создадим сами. Но сначала чекнем пропущенные значения.

In [ ]:
df.isna().mean()

InvoiceNo      0.000000
StockCode      0.000000
Description    0.002683
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
CustomerID     0.249267
Country        0.000000
dtype: float64

У нас куча юзеров не заполнена. Если делать груп бай по NaN, то NaN просто не учитывается. Нам это не подходит. По этому заполним пропущенных юзеров значением -999999. Считай это технической заменой, чтобы этот юзер отображался в сводниках.

In [ ]:
df['CustomerID'] = df['CustomerID'].fillna(-999999)

In [ ]:
user = df.groupby(['CustomerID'])['InvoiceDate'].min().reset_index()

In [ ]:
user.head()

,CustomerID,InvoiceDate
0,-999999.0,2010-12-01 11:52:00
1,12346.0,2011-01-18 10:01:00
2,12347.0,2010-12-07 14:57:00
3,12348.0,2010-12-16 19:09:00
4,12349.0,2011-11-21 09:51:00


In [ ]:
user.columns = ['CustomerID','reg_date']

In [ ]:
user['reg_month'] = user['reg_date'].values.astype('datetime64[M]')

Темная магия). На самом деле просто первая выдача гугла по запросу "pandas how to get first day of month" https://stackoverflow.com/questions/45304531/extracting-the-first-day-of-month-of-a-datetime-type-column-in-pandas



In [ ]:
user

,CustomerID,reg_date,reg_month
0,-999999.0,2010-12-01 11:52:00,2010-12-01
1,12346.0,2011-01-18 10:01:00,2011-01-01
2,12347.0,2010-12-07 14:57:00,2010-12-01
3,12348.0,2010-12-16 19:09:00,2010-12-01
4,12349.0,2011-11-21 09:51:00,2011-11-01
...,...,...,...
4368,18280.0,2011-03-07 09:52:00,2011-03-01
4369,18281.0,2011-06-12 10:53:00,2011-06-01
4370,18282.0,2011-08-05 13:35:00,2011-08-01
4371,18283.0,2011-01-06 14:14:00,2011-01-01


# 2. Сгенерируй таблицу где на каждого юзера будет каждый месяц


In [ ]:
min_month = df['InvoiceDate'].values.astype('datetime64[M]').min()

In [ ]:
max_month = df['InvoiceDate'].values.astype('datetime64[M]').max()

In [ ]:
(min_month, max_month)

(numpy.datetime64('2010-12'), numpy.datetime64('2011-12'))

In [ ]:
pd.date_range(min_month,max_month,freq='MS')

DatetimeIndex(['2010-12-01', '2011-01-01', '2011-02-01', '2011-03-01',
               '2011-04-01', '2011-05-01', '2011-06-01', '2011-07-01',
               '2011-08-01', '2011-09-01', '2011-10-01', '2011-11-01',
               '2011-12-01'],
              dtype='datetime64[ns]', freq='MS')

In [ ]:
dr = pd.DataFrame(pd.date_range(min_month,max_month,freq='MS'))

In [ ]:
dr

,0
0,2010-12-01
1,2011-01-01
2,2011-02-01
3,2011-03-01
4,2011-04-01
5,2011-05-01
6,2011-06-01
7,2011-07-01
8,2011-08-01
9,2011-09-01


In [ ]:
dr.columns = ['month']

Теперь надо сделать крос джоин. Должно получиться


In [ ]:
len(user)*len(dr)

56849

строк. В пандас крос джоин делается так

In [ ]:
dr['key'] = 1
user['key'] = 1

In [ ]:
report = dr.merge(user,on='key')

In [ ]:
report.head()

,month,key,CustomerID,reg_date,reg_month
0,2010-12-01,1,-999999.0,2010-12-01 11:52:00,2010-12-01
1,2010-12-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
2,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01
3,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01
4,2010-12-01,1,12349.0,2011-11-21 09:51:00,2011-11-01


In [ ]:
len(report)

56849

Как и ожидали. Но у нас есть пользователи которые пришли после минимальной даты (2010-12) и тупо хранить по ним строчки за тот период пока их с нами не было

In [ ]:
len(report[report['month']>=report['reg_month']])

37449

In [ ]:
report = report[report['month']>=report['reg_month']]

Посмотрим на отдельного юзера

In [ ]:
report[report['CustomerID'] == 12346.0]

,month,key,CustomerID,reg_date,reg_month
4374,2011-01-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
8747,2011-02-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
13120,2011-03-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
17493,2011-04-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
21866,2011-05-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
26239,2011-06-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
30612,2011-07-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
34985,2011-08-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
39358,2011-09-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
43731,2011-10-01,1,12346.0,2011-01-18 10:01:00,2011-01-01


In [ ]:
report[report['CustomerID'] == 12448.0]

,month,key,CustomerID,reg_date,reg_month
43814,2011-10-01,1,12448.0,2011-10-26 13:46:00,2011-10-01
48187,2011-11-01,1,12448.0,2011-10-26 13:46:00,2011-10-01
52560,2011-12-01,1,12448.0,2011-10-26 13:46:00,2011-10-01


Теперь можно джойнить инфу по продажам. Но сначала

# 3. Сгруппируй данные по продажам перед тем как джойнить их к отчету.
 
Посчитаем суммарные продажи на каждый месяц.


In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df['month'] = df['InvoiceDate'].values.astype('datetime64[M]')

In [ ]:
df['revenue'] = df['Quantity'] * df['UnitPrice']

Там есть отрицательные значения по количеству -- наверное это возвраты. Но для этого туториала это не важно, мы тут учимся общим принципам джойнинья и составлению отчетов

In [ ]:
sales_month = df.groupby(['CustomerID','month'])[['revenue']].agg(['sum']).reset_index()

In [ ]:
sales_month.head()

,CustomerID,month,revenue
,,,sum
0,-999999.0,2010-12-01,194353.00
1,-999999.0,2011-01-01,84925.88
2,-999999.0,2011-02-01,61516.50
3,-999999.0,2011-03-01,103302.47
4,-999999.0,2011-04-01,67159.27


In [ ]:
sales_month.columns = ['CustomerID','month','revenue']

In [ ]:
sales_month.head()

,CustomerID,month,revenue
0,-999999.0,2010-12-01,194353.00
1,-999999.0,2011-01-01,84925.88
2,-999999.0,2011-02-01,61516.50
3,-999999.0,2011-03-01,103302.47
4,-999999.0,2011-04-01,67159.27


И здесь обязательно проверка по то тоталу, общей сумме по ревеню. Можешь называть это контрольной суммой. Сколько бы таблиц ты не делал тоталы всегда должны сходится. Если не проверил бьются ли тоталы, но ты не имеешь права отправлять свой отчет никому. Засмеют.

In [ ]:
df['revenue'].sum()

9747747.933999998

In [ ]:
sales_month['revenue'].sum()

9747747.934000025

Сходится. То что там не бьются далекие цифры полсе запятой это ок. https://stackoverflow.com/a/11873114/4527289

# 4. Джойним к отчету

In [ ]:
report = report.merge(sales_month,how='left',on=['CustomerID','month'])

Очень важно использовать лефт джоин! Когда используешь лефт джоин, то остануться все строки из таблицы report. И мы сможем увидеть месяца, когда у клиента не было активности

In [ ]:
report.head()

,month,key,CustomerID,reg_date,reg_month,revenue
0,2010-12-01,1,-999999.0,2010-12-01 11:52:00,2010-12-01,194353.00
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79
2,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01,892.80
3,2010-12-01,1,12370.0,2010-12-14 12:58:00,2010-12-01,1868.02
4,2010-12-01,1,12377.0,2010-12-20 09:37:00,2010-12-01,1001.52


После джойна тоже проверка по котрольной сумме. Обязательно.

In [ ]:
df['revenue'].sum()

9747747.933999998

In [ ]:
report['revenue'].sum()

9747747.934000023

И так же очень полезна проверка на вменяемость с помощью глаз головы. На англисйском это называется sanity check. Посмотрим на отдельного клиента.

In [ ]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN


Вроде ок. Видно, что активность этого клиента нерегулярная! Например у него есть перерыв в покупках после 2011-01-01 на два месяца. 

# 5. Базовые метрики

С помощью этой таблицы можно расчитать кучу всего полезного

Продажи на каждый месяц

In [ ]:
report.groupby('month')[['revenue']].agg('sum')

,revenue
month,
2010-12-01,748957.020
2011-01-01,560000.260
2011-02-01,498062.650
2011-03-01,683267.080
2011-04-01,493207.121
2011-05-01,723333.510
2011-06-01,691123.120
2011-07-01,681300.111
2011-08-01,682680.510


Добавим флаг юзера, нового юзера, и активного юзера

In [ ]:
report['user'] = 1
report['new'] = (report['reg_month'] == report['month']) * 1
report['active'] = (report['revenue'] > 0) * 1

In [ ]:
report.head()

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active
0,2010-12-01,1,-999999.0,2010-12-01 11:52:00,2010-12-01,194353.00,1,1,1
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1
2,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01,892.80,1,1,1
3,2010-12-01,1,12370.0,2010-12-14 12:58:00,2010-12-01,1868.02,1,1,1
4,2010-12-01,1,12377.0,2010-12-20 09:37:00,2010-12-01,1001.52,1,1,1


In [ ]:
report.groupby('month')[['revenue','user','new','active']].agg('sum')

,revenue,user,new,active
month,,,,
2010-12-01,748957.020,949,949,882
2011-01-01,560000.260,1370,421,737
2011-02-01,498062.650,1750,380,754
2011-03-01,683267.080,2190,440,969
2011-04-01,493207.121,2489,299,851
2011-05-01,723333.510,2768,279,1053
2011-06-01,691123.120,3003,235,987
2011-07-01,681300.111,3194,191,943
2011-08-01,682680.510,3361,167,931


Средние продажи на клиента

In [ ]:
t = report.groupby('month')[['revenue','user','new','active']].agg('sum')

In [ ]:
t['sales_per_customer'] = t['revenue'] / t['active']

In [ ]:
t

,revenue,user,new,active,sales_per_customer
month,,,,,
2010-12-01,748957.020,949,949,882,849.157619
2011-01-01,560000.260,1370,421,737,759.837531
2011-02-01,498062.650,1750,380,754,660.560544
2011-03-01,683267.080,2190,440,969,705.125986
2011-04-01,493207.121,2489,299,851,579.561834
2011-05-01,723333.510,2768,279,1053,686.926410
2011-06-01,691123.120,3003,235,987,700.226059
2011-07-01,681300.111,3194,191,943,722.481560
2011-08-01,682680.510,3361,167,931,733.276595


# 6. Отток

Таким же образом можно рассчитать метрики по оттоку. Например количество клиентов которые не активны 1,2,3,...,n месяцев. Я буду проверять всю логику на одном клиенте.

In [ ]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0


На каждого клиента я хочу посмотреть был ли он активен в предыдущем месяце.


In [ ]:
report['active_prev'] = (report.sort_values(by=['month'], ascending=True)
                       .groupby(['CustomerID'])['active'].shift(1))

Вот эта дичь это типа как оконная функция в sql. У меня много клиентов и мне надо как-то объяснить пандасу, чтобы он рассчитал активность в предыдущем месяце для каждого клиента в отдельности -- для этого тут groupby. Также я хочу чтобы месяцы шли по порядку -- для этого sort_values. Ну и shift уже дает мне активность в предыдущем месяце

In [ ]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0


Похоже на правду. Сравни active с active_prev. Теперь я хочу понять в какие месяца клиент менял статус с активного на неактивного и наоборот.

In [ ]:
report['change_status'] = (report['active'] != report['active_prev']) * 1

In [ ]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev,change_status
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN,1
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0,0
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0,0
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0,1
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0,1
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0,1
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1


Супер. Теперь я хочу назначить айдишку каждому изменению статуса. И здесь прикольный трюк с cumsum. Посмотри на change_status. Если сделать по нему cumsum то эта сумма будет увеличиваться на один каждый раз когда клиент меняет статус Это и будет айдишка. И тут я снова использую что-то вроде оконной функции, чтобы эта айдишка была своя у каждого клиента.

In [ ]:
report['session_id'] = (report.sort_values(by=['month'], ascending=True)
                       .groupby(['CustomerID'])['change_status'].cumsum())

In [ ]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev,change_status,session_id
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN,1,1
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0,0,1
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,2
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0,0,2
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0,1,3
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,4
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0,1,5
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,6
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0,1,7
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,8


Отлично. Найди session_id 2. Видно, что в этой сессии клиент был неактивен два месяца. Теперь я могу просуммировать количество неактивных месяцов на каждую сессию

In [ ]:
report['inactive'] = (report['active'] == 0) * 1

In [ ]:
report['month_inactive'] = (report.sort_values(by=['month'], ascending=True)
                       .groupby(['CustomerID','session_id'])['inactive'].cumsum())

Посмотри на строку номер 4 (индекс 4067). В month_inactive получилось правильное количество месяцев когда клиент был неактивным. Проверь глазами.

In [ ]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev,change_status,session_id,inactive,month_inactive
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN,1,1,0,0
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0,0,1,0,0
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,2,1,1
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0,0,2,1,2
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0,1,3,0,0
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,4,1,1
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0,1,5,0,0
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,6,1,1
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0,1,7,0,0
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,8,1,1


Теперь посчитаем саму метрику. Например количество клиентов которые были в оттоке (неактивны) в течении двух месяцев

In [ ]:
report['churn_2m'] = (report['month_inactive'] >= 2) * 1

In [ ]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev,change_status,session_id,inactive,month_inactive,churn_2m
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN,1,1,0,0,0
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0,0,1,0,0,0
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,2,1,1,0
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0,0,2,1,2,1
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0,1,3,0,0,0
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,4,1,1,0
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0,1,5,0,0,0
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,6,1,1,0
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0,1,7,0,0,0
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,8,1,1,0


Отлично. Кстати с помощью этого метода (cumsum), можно считать пользовательские сессии -- может быть понадобиться тебе в будущем. Теперь сгруппируем на месяц.

In [ ]:
t = report.groupby('month')[['revenue','user','new','active','churn_2m']].agg('sum')

In [ ]:
t

,revenue,user,new,active,churn_2m
month,,,,,
2010-12-01,748957.020,949,949,882,0
2011-01-01,560000.260,1370,421,737,55
2011-02-01,498062.650,1750,380,754,509
2011-03-01,683267.080,2190,440,969,749
2011-04-01,493207.121,2489,299,851,966
2011-05-01,723333.510,2768,279,1053,1225
2011-06-01,691123.120,3003,235,987,1367
2011-07-01,681300.111,3194,191,943,1619
2011-08-01,682680.510,3361,167,931,1871


Так же неплохо бы получить этот показатель в процентах. Выбор знаменателя зависит от того, как ты хочешь считать этот процент. Здесь я просто поделю их на текущее количество пользователей в этом месяце. Но можно например делать на количество пользователей два месяца назад (чтобы в знаменатель не попали новые клиенты)

In [ ]:
t['churn_2m%'] = t['churn_2m'] / t['user']

In [ ]:
t

,revenue,user,new,active,churn_2m,churn_2m%
month,,,,,,
2010-12-01,748957.020,949,949,882,0,0.000000
2011-01-01,560000.260,1370,421,737,55,0.040146
2011-02-01,498062.650,1750,380,754,509,0.290857
2011-03-01,683267.080,2190,440,969,749,0.342009
2011-04-01,493207.121,2489,299,851,966,0.388108
2011-05-01,723333.510,2768,279,1053,1225,0.442558
2011-06-01,691123.120,3003,235,987,1367,0.455211
2011-07-01,681300.111,3194,191,943,1619,0.506888
2011-08-01,682680.510,3361,167,931,1871,0.556680


Это очень надежный метод для отчетов: ежедневный, еженедельный, ежемесячный. Если ты сгенеришь все возможные даты заранее, то никогда ничего не упустишь. Единственный минув этого метода -- он не маштабируемый. С миллионом клиентов кросс джоин захлебнется. И даже в sql он тоже не маштабируемый.

https://www.glebmikhaylov.com/